In [274]:
import sys
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import os
import itertools
import socket
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import time
import holidays
import pycountry

import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from IPython.display import display
from IPython.display import clear_output

In [275]:
# from catboost import CatBoostClassifier
# from catboost.utils import get_gpu_device_count

# # Check available GPUs
# gpu_count = get_gpu_device_count()
# if gpu_count > 0:
#     print(f"GPU is available with {gpu_count} GPU(s).")
# else:
#     print("No GPU detected. Ensure your setup is correct.")

# # Test CatBoost with GPU
# try:
#     model = CatBoostClassifier(task_type="GPU", devices='0')  # Specify GPU
#     device = 'cuda'
#     print("CatBoost can use the GPU.")
        
# except Exception as e:
#     device = 'cpu'
#     print(f"Error: {e}")

In [276]:
# Declare important variables
train_path = 'train.csv'
test_path = 'test.csv'
target_variable = 'num_sold'
SEED = 69
skip_hypertuning = True
cat_encoder_type = 'OneHotEncoder' #OneHotEncoder, FrequencyEncoder, None

In [277]:
def load_dataframes(path1: str, path2: str, format1: str = "csv", format2: str = "csv") -> tuple:

    loaders = {
        "csv": pd.read_csv,
        "excel": pd.read_excel,
        "json": pd.read_json,
    }

    if format1 not in loaders or format2 not in loaders:
        raise ValueError("Unsupported format. Supported formats: 'csv', 'excel', 'json'.")

    # Load the dataframes using appropriate loaders
    df1 = loaders[format1](path1)
    df2 = loaders[format2](path2)

    print(f"Loading data from {os.getcwd()}")
    
    return df1, df2

In [278]:
df_train, df_test = load_dataframes(train_path,test_path)

df_train = df_train.rename(columns={target_variable: 'y'})

Loading data from C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\Kaggle\Playground Series\s5e1 - Sticker Sales


In [279]:
#plot_kde_grid(df_train,n_cols=4,figsize=(12,8))

In [280]:
#plot_kde_grid(df_solve,n_cols=4,figsize=(9,6))

In [281]:
def get_num_cols(df: pd.DataFrame):
    num_cols = df.select_dtypes(include=['number']).columns.to_list()
    
    return num_cols

def get_cat_cols(df: pd.DataFrame):
    cat_cols = df.select_dtypes(include=['object', 'string', 'category']).columns.tolist()  

    return cat_cols

In [282]:
def describe_dataframe(df_to_use: pd.DataFrame, name: str = 'DataFrame', nrows: int = 3, plots: bool = False, info: bool = True) -> None:
    '''
    Function to describe the DataFrame with summary statistics, missing value count,
    unique value count, and duplicate count. It also displays plots for missing and unique values.
    
    Parameters:
    - df: The DataFrame to describe.
    - name: The name to display in the summary (default is 'DataFrame').
    - nrows: The number of rows to display from the top and bottom (default is 3).
    - plots: Whether to display bar plots for missing and unique values (default is False).
    - info: Whether to display the styled DataFrame (default is True).
    '''

    df = df_to_use.copy()   
    
    # Create summary DataFrame
    inf = pd.DataFrame(df.dtypes).reset_index().rename(columns={'index':'column', 0:'type'})
    
    # Missing values
    df_missing = pd.DataFrame(df.isnull().sum()).reset_index().rename(columns={'index':'column', 0:'missing'})
    df_missing['pct_missing'] = (df_missing['missing'] / df.shape[0]) * 100
    
    # Unique values
    df_unique = pd.DataFrame(df.nunique()).reset_index().rename(columns={'index':'column', 0:'unique'})
    
    # Combine summary information
    inf['missing'] = df_missing['missing']
    inf['pct_missing'] = df_missing['pct_missing']
    inf['unique'] = df_unique['unique']
    inf['duplicate'] = df.duplicated().sum()
    inf['count'] = df.shape[0]

    # Descriptive statistics
    desc = pd.DataFrame(df.describe(include='all').transpose())
    if 'min' in desc.columns.tolist():
        inf['min'] = desc['min'].values
        inf['max'] = desc['max'].values
        inf['avg'] = desc['mean'].values
        inf['std dev'] = desc['std'].values
    if 'top' in desc.columns.tolist():
        inf['top value'] = desc['top'].values
        inf['Freq'] = desc['freq'].values 
    
    # Display styled DataFrame
    if info:
        display(inf.style.background_gradient(subset=['missing','pct_missing'], cmap='Reds').background_gradient(subset='unique', cmap='Greens'))

    if nrows != 0 :
        # Display top and bottom nrows of the DataFrame
        print(f"\n---------- {name} Overview ----------:")
        print(f"{name} has {df.shape[0]} rows and {df.shape[1]} columns\n")
        display(df.head(nrows))
        display(df.tail(nrows))
    
    # Plot missing values if any
    if plots and df_missing['missing'].sum() > 0:
        fig, ax = plt.subplots(1, 1, figsize=(24, 5))
        sns.barplot(df_missing[df_missing['missing'] > 0], x='column', y='missing', ax=ax)
        ax.set_title(f'{name} missing Values') 
        ax.bar_label(ax.containers[0])
        plt.tight_layout()
        plt.show()
    
        # Plot unique values
        fig, ax = plt.subplots(1, 1, figsize=(24, 5))
        sns.barplot(df_unique[df_unique['unique'] > 0], x='column', y='unique', ax=ax)
        ax.set_title(f'{name} Unique Values')
        ax.bar_label(ax.containers[0])
        plt.tight_layout()
        plt.show()

In [283]:
describe_dataframe(df_train, name='Insurance Data', nrows=0, plots=False, info=True)

,column,type,missing,pct_missing,unique,duplicate,count,min,max,avg,std dev,top value,Freq
0,id,int64,0,0.000000,230130,0,230130,0.000000,230129.000000,115064.500000,66432.953062,nan,nan
1,date,object,0,0.000000,2557,0,230130,nan,nan,nan,nan,2010-01-01,90
2,country,object,0,0.000000,6,0,230130,nan,nan,nan,nan,Canada,38355
3,store,object,0,0.000000,3,0,230130,nan,nan,nan,nan,Discount Stickers,76710
4,product,object,0,0.000000,5,0,230130,nan,nan,nan,nan,Holographic Goose,46026
5,y,float64,8871,3.854778,4037,0,230130,5.000000,5939.000000,752.527382,690.165445,nan,nan


In [284]:
def clean_dataframe(df_to_use: pd.DataFrame, impute_num_nulls: bool = True, fill_cat_nulls: bool = True) -> None:
    df = df_to_use.copy()

    # Convert date column from object to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter columns where the type is either 'float' or 'int' and there are missing values
    numeric_cols = df.select_dtypes(include=['number']).columns
    missing_numeric_columns = [
        col for col in numeric_cols if df[col].isnull().sum() > 0
    ]

    # Target field 'num_sold' is missing some values, we do not want to impute them
    if 'y' in missing_numeric_columns:
        missing_numeric_columns.remove('y')
    
    # Fill nulls in numeric columns with the median
    if impute_num_nulls:       
        for column in missing_numeric_columns:
            mdn = df[column].median()
            df[column] = df[column].fillna(mdn)

    
    # Get category columns
    if fill_cat_nulls:
        cat_cols = df.select_dtypes(include=['object', 'string','category']).columns.tolist()  
        # Fill missing values in object columns
        for column in cat_cols:
            df[column] = df[column].fillna('None')
    
    return df

In [285]:
df_train = clean_dataframe(df_train, impute_num_nulls = True, fill_cat_nulls=False)
df_test = clean_dataframe(df_test, impute_num_nulls = True, fill_cat_nulls=False)

In [286]:
describe_dataframe(df_train, name='Insurance Data', nrows=0, plots=False, info=True)

C:\Users\zchodaniecky\AppData\Local\Temp\ipykernel_21380\190788460.py:34: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  desc = pd.DataFrame(df.describe(include='all').transpose())


,column,type,missing,pct_missing,unique,duplicate,count,min,max,avg,std dev,top value,Freq
0,id,int64,0,0.000000,230130,0,230130,0.000000,230129.000000,115064.500000,66432.953062,nan,nan
1,date,datetime64[ns],0,0.000000,2557,0,230130,nan,nan,nan,nan,2010-01-01 00:00:00,90
2,country,object,0,0.000000,6,0,230130,nan,nan,nan,nan,Canada,38355
3,store,object,0,0.000000,3,0,230130,nan,nan,nan,nan,Discount Stickers,76710
4,product,object,0,0.000000,5,0,230130,nan,nan,nan,nan,Holographic Goose,46026
5,y,float64,8871,3.854778,4037,0,230130,5.000000,5939.000000,752.527382,690.165445,nan,nan


In [287]:
df_train = df_train.dropna(subset='y')

In [288]:
def get_holiday_name(country_code, date_obj):
    try:
        country_holiday = holidays.CountryHoliday(country_code, years=date_obj.year)
        return country_holiday.get(date_obj)
    except Exception as e:
        print(f"Error for country code {country_code} and date {date_obj}: {e}")
        return 'Invalid Holiday'
    return country_holiday.get(date_obj)

def get_country_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_2  
    except KeyError:
        print(f"Unknown Country: {country_name}")
        return None

def get_holiday_for_row(row):
    country_code = get_country_code(row['country'])
    if country_code is None:
        return 'Unknown Country'     
    try:
        date_obj = row['date']
    except ValueError:
        print(f"Invalid Date: {row['date']}")
        return 'Invalid Date'

    return get_holiday_name(country_code, date_obj)


for df in [df_train, df_test]:
    df['holiday'] = df.apply(get_holiday_for_row, axis=1)


In [289]:
for df in [df_train, df_test]:
    df['Year'] = df['date'].dt.year
    df['Quarter'] = df['date'].dt.quarter
    df['Month'] = df['date'].dt.month
    df['Day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.day_name()
    df['day_of_year'] = df['date'].dt.dayofyear
    df['week_of_year'] = df['date'].dt.isocalendar().week

    df['day_sin'] = np.sin(2 * np.pi * df['Day'] / 365.0)
    df['day_cos'] = np.cos(2 * np.pi * df['Day'] / 365.0)
    df['month_sin'] = np.sin(2 * np.pi * df['Month'] / 12.0)
    df['month_cos'] = np.cos(2 * np.pi * df['Month'] / 12.0)
    df['year_sin'] = np.sin(2 * np.pi * df['Year'] / 7.0)
    df['year_cos'] = np.cos(2 * np.pi * df['Year'] / 7.0)
    df['Group']=(df['Year']-2010)*48+df['Month']*4+df['Day']//7


    df['Quarter'] = df['Quarter'].astype('str')
    df['Month'] = df['Month'].astype('str')
    df['day_of_week'] = df['day_of_week'].astype('str')
    df['day_of_year'] = df['day_of_week'].astype('str')
    df['week_of_year'] = df['week_of_year'].astype('str')


In [290]:
RMV = ['y','id','date']
FEATURES = [c for c in df.columns if not c in RMV]
combined = pd.concat([df_train,df_solve],axis=0,ignore_index=True)

CATS = []
HIGH_CARDINALITY = []
print(f"THE {len(FEATURES)} BASIC FEATURES ARE:")

for c in FEATURES:
    ftype = "numerical"
    if combined[c].dtype=="object":
        CATS.append(c)
        combined[c] = combined[c].fillna("NAN")
        combined[c],_ = combined[c].factorize()
        combined[c] -= combined[c].min()
        ftype = "categorical"
    if combined[c].dtype=="int64":
        combined[c] = combined[c].astype("int32")
    elif combined[c].dtype=="float64":
        combined[c] = combined[c].astype("float32")

    n = combined[c].nunique()
    print(f"{c} ({ftype}) with {n} unique values")
    if n>=9: HIGH_CARDINALITY.append(c)
    
df_train = combined.iloc[:len(df_train)].copy()
df_test = combined.iloc[len(df_train):].reset_index(drop=True).copy()

print("\nTHE FOLLOWING HAVE 9 OR MORE UNIQUE VALUES:", HIGH_CARDINALITY )


df_train = df_train.drop(['date','id'], axis=1)
df_test = df_test.drop(['date','id'], axis=1)


THE 18 BASIC FEATURES ARE:
country (categorical) with 12 unique values
store (categorical) with 6 unique values
product (categorical) with 10 unique values
holiday (categorical) with 155 unique values
Year (numerical) with 10 unique values
Quarter (categorical) with 8 unique values
Month (categorical) with 24 unique values
Day (numerical) with 31 unique values
day_of_week (categorical) with 14 unique values
day_of_year (categorical) with 14 unique values
week_of_year (categorical) with 105 unique values
day_sin (numerical) with 31 unique values
day_cos (numerical) with 31 unique values
month_sin (numerical) with 8 unique values
month_cos (numerical) with 8 unique values
year_sin (numerical) with 7 unique values
year_cos (numerical) with 4 unique values
Group (numerical) with 481 unique values

THE FOLLOWING HAVE 9 OR MORE UNIQUE VALUES: ['country', 'product', 'holiday', 'Year', 'Month', 'Day', 'day_of_week', 'day_of_year', 'week_of_year', 'day_sin', 'day_cos', 'Group']


In [ ]:
describe_dataframe(df_train, name='Sales Data', nrows=0, plots=False, info=True)

In [ ]:
# sns.set(rc={'figure.figsize':(24,8)})
# ax=sns.lineplot(data=df_train,x='date',y='y',hue='product')
# ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);`

In [ ]:
# sns.set(rc={'figure.figsize':(24,8)})
# ax=sns.lineplot(data=df_train,x='Day of Year',y='y')
# ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
# sns.set(rc={'figure.figsize':(24,8)})
# ax=sns.lineplot(data=df_train,x='Week of Year',y='y')
# ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
# sns.set(rc={'figure.figsize':(24,8)})
# ax=sns.lineplot(data=df_train,x='Month',y='y')
# ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
# sns.set(rc={'figure.figsize':(24,8)})
# ax=sns.lineplot(data=df_train,x='Year',y='y', hue='country')
# ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
# sns.set(rc={'figure.figsize':(24,8)})
# ax=sns.lineplot(data=df_train,x='Day of Year Sin',y='y')
# ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
def target_encode(train, valid, test, col, target="y", kfold=5, smooth=20, agg="mean"):
             
    train['kfold'] = ((train.index) % kfold)
    col_name = '_'.join(col)
    train[f'TE_{agg.upper()}_' + col_name] = 0.
    for i in range(kfold):
        
        df_tmp = train[train['kfold']!=i]       
        if agg=="mean": mn = train[target].mean()
        elif agg=="median": mn = train[target].median()
        elif agg=="min": mn = train[target].min()
        elif agg=="max": mn = train[target].max()
        elif agg=="nunique": mn = 0
        df_tmp = df_tmp[col + [target]].groupby(col).agg([agg, 'count']).reset_index()
        df_tmp.columns = col + [agg, 'count']
        if agg=="nunique":
            df_tmp['TE_tmp'] = df_tmp[agg] / df_tmp['count']
        else:
            df_tmp['TE_tmp'] = ((df_tmp[agg]*df_tmp['count'])+(mn*smooth)) / (df_tmp['count']+smooth)
        df_tmp_m = train[col + ['kfold', f'TE_{agg.upper()}_' + col_name]].merge(df_tmp, how='left', left_on=col, right_on=col)
        df_tmp_m.loc[df_tmp_m['kfold']==i, f'TE_{agg.upper()}_' + col_name] = df_tmp_m.loc[df_tmp_m['kfold']==i, 'TE_tmp']
        train[f'TE_{agg.upper()}_' + col_name] = df_tmp_m[f'TE_{agg.upper()}_' + col_name].fillna(mn).values  
    
    df_tmp = train[col + [target]].groupby(col).agg([agg, 'count']).reset_index()
    if agg=="mean": mn = train[target].mean()
    elif agg=="median": mn = train[target].median()
    elif agg=="min": mn = train[target].min()
    elif agg=="max": mn = train[target].max()
    elif agg=="nunique": mn = 0
    df_tmp.columns = col + [agg, 'count']
    
    if agg=="nunique":
        df_tmp['TE_tmp'] = df_tmp[agg] / df_tmp['count']
    else:
        df_tmp['TE_tmp'] = ((df_tmp[agg]*df_tmp['count'])+(mn*smooth)) / (df_tmp['count']+smooth)
        
    valid_col_df = valid[col]
    df_tmp_m = valid_col_df.merge(df_tmp, how='left', left_on=col, right_on=col).reset_index(drop=True)
    valid[f'TE_{agg.upper()}_' + col_name] = df_tmp_m['TE_tmp'].fillna(mn).values
    valid[f'TE_{agg.upper()}_' + col_name] = valid[f'TE_{agg.upper()}_' + col_name].astype("float32")
    
    test_col_df = test[col]
    df_tmp_m = test_col_df.merge(df_tmp, how='left', left_on=col, right_on=col).reset_index(drop=True)
    test[f'TE_{agg.upper()}_' + col_name] = df_tmp_m['TE_tmp'].fillna(mn).values
    test[f'TE_{agg.upper()}_' + col_name] = test[f'TE_{agg.upper()}_' + col_name].astype("float32")

    train = train.drop('kfold', axis=1)
    train[f'TE_{agg.upper()}_' + col_name] = train[f'TE_{agg.upper()}_' + col_name].astype("float32")

    return(train, valid, test)

In [ ]:
# lists2 = [['Annual Income', 'Health Score'], ['Credit Score', 'Health Score'], ['Customer Feedback', 'Gender', 'Marital Status', 'Occupation', 'Smoking Status', 'year'], ['Exercise Frequency', 'Health Score'], ['Health Score', 'Marital Status'], ['Education Level', 'Gender', 'Health Score'], ['Health Score', 'Occupation'], ['Age', 'Health Score'], ['Health Score', 'dow'], ['Age', 'Exercise Frequency', 'Location'], ['Health Score', 'Smoking Status', 'month'], ['Health Score', 'Location', 'Policy Type'], ['Health Score', 'Insurance Duration'], ['Health Score', 'Number of Dependents'], ['Customer Feedback', 'Exercise Frequency', 'Previous Claims', 'Property Type', 'dow'], ['Customer Feedback', 'Health Score'], ['Health Score', 'Property Type'], ['Health Score', 'day', 'seconds'], ['Health Score', 'year'], ['Age', 'Gender', 'Insurance Duration', 'year']]
# print(f"We have {len(lists2)} powerful combination of columns!")
# print(lists2)
lists2 = []

In [ ]:
import time
from xgboost import XGBRegressor
import xgboost as xgb, time
print(f"Using XGBoost version",xgb.__version__)


start = time.time()

FOLDS = 20
from sklearn.model_selection import KFold
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

oof = np.zeros(len(df_train))
pred = np.zeros(len(df_test))

for i, (train_index, test_index) in enumerate(kf.split(df_train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)
    
    x_train = df_train.loc[train_index,FEATURES+["y"] ].copy()
    y_train = df_train.loc[train_index,"y"]
    x_valid = df_train.loc[test_index,FEATURES].copy()
    y_valid = df_train.loc[test_index,"y"]
    x_test = df_test[FEATURES].copy()

    start = time.time()
    print(f"FEATURE ENGINEER {len(FEATURES)} COLUMNS and {len(lists2)} GROUPS: ",end="")
    for j,f in enumerate(FEATURES+lists2):

        if j<len(FEATURES): c = [f]
        else: c = f 
        print(f"({j+1}){c}",", ",end="")

        # LOW CARDINALITY FEATURES - TARGET ENCODE MEAN AND MEDIAN
        x_train, x_valid, x_test = target_encode(x_train, x_valid, x_test, c, smooth=20, agg="mean")
        x_train, x_valid, x_test = target_encode(x_train, x_valid, x_test, c, smooth=0, agg="median")

        # HIGH CARDINALITY FEATURES - TE MIN, MAX, NUNIQUE and CE
        if (j>=len(FEATURES)) | (c[0] in HIGH_CARDINALITY):
            x_train, x_valid, x_test = target_encode(x_train, x_valid, x_test, c, smooth=0, agg="min")
            x_train, x_valid, x_test = target_encode(x_train, x_valid, x_test, c, smooth=0, agg="max")
            x_train, x_valid, x_test = target_encode(x_train, x_valid, x_test, c, smooth=0, agg="nunique")
    
            # COUNT ENCODING (USING COMBINED TRAIN TEST)
            tmp = combined.groupby(c).y.count()
            nm = f"CE_{'_'.join(c)}"; tmp.name = nm
            x_train = x_train.merge(tmp, on=c, how="left")
            x_valid = x_valid.merge(tmp, on=c, how="left")
            x_test = x_test.merge(tmp, on=c, how="left")
            x_train[nm] = x_train[nm].astype("int32")
            x_valid[nm] = x_valid[nm].astype("int32")
            x_test[nm] = x_test[nm].astype("int32")
            
    end = time.time()
    elapsed = end-start
    print(f"Feature engineering took {elapsed:.1f} seconds")
    x_train = x_train.drop("y",axis=1)
    
    model = XGBRegressor(
        device="cuda",
        max_depth=8, 
        colsample_bytree=0.9, 
        subsample=0.9, 
        n_estimators=2_000, 
        learning_rate=0.01, 
        early_stopping_rounds=25,  
        eval_metric="rmse",
    )
    model.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],   
        verbose=100
    )

    # INFER OOF
    oof[test_index] = model.predict(x_valid)
    # INFER TEST
    pred += model.predict(x_test)

    m = np.sqrt(np.mean( (y_valid.to_numpy() - oof[test_index])**2.0 )) 
    print(f" => Fold {i+1} RMSLE = {m:.5f}")
    
# COMPUTE AVERAGE TEST PREDS
pred /= FOLDS

In [ ]:
# #####
# # Try Bayesian Optimization
# #### 
# from sklearn.model_selection import GridSearchCV
# from skopt import BayesSearchCV
# from xgboost import XGBRegressor

# # Hyperparameters for BayesSearchCV tuning
# search_spaces = {
#     'n_estimators': (10, 200),      
#     'max_depth': (3, 10),            
#     'reg_alpha': (0.001, 0.2, 'log-uniform'),  
#     'reg_lambda': (0.1, 100, 'log-uniform')     
# }

# # Leave some cores available
# available_cores = os.cpu_count()
# n_cores = max(1, available_cores - 3)  # Leave 3 cores free


# xgb = XGBRegressor(random_state=SEED)

# # Set up BayesSearchCV for hyperparameter tuning
# search = BayesSearchCV(
#     estimator=xgb,
#     search_spaces=search_spaces,
#     n_iter=50,          # Number of iterations for optimization
#     cv=5,               # k-fold cross-validation
#     verbose=1,          # Display detailed logs
#     scoring="neg_root_mean_squared_error",
#     random_state=42     
# )

# if skip_hypertuning:

#     best_params = {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1200}

# else:
#     start_time = time.time()
    
#     grid_search.fit(X_train_prepared, y_log_train)
    
#     end_time = time.time()  
    
#     best_params = grid_search.best_params_
#     best_rmse = (-grid_search.best_score_) ** 0.5
    
#     print("Best parameters:",best_params)
#     print("Best RMSE:", best_rmse)

#     # Calculate elapsed time in minutes
#     elapsed_time_minutes = (end_time - start_time) / 60
#     print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")

In [ ]:

# # Initialize the XGBRegressor
# model = XGBRegressor(
#     **best_params,
#     random_state=SEED,        # Ensure reproducibility
#     objective='reg:squarederror'
# )

# # Train the model
# model.fit(
#     X_train_prepared, 
#     y_train, 
#     eval_set=[(X_val_prepared, y_val)],
#     verbose=0
# )

In [ ]:
from xgboost import plot_importance

#Plot feature importance
plot_importance(model, importance_type='gain', max_num_features=min(X_train_prepared.shape[1], 25))  
# 'weight', 'gain', 'cover' are possible importance types
plt.show()

In [ ]:
# Predict on test set
y_test_pred = model.predict(X_test_prepared)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# Calculate RMSE
mape = mean_absolute_percentage_error(y_test, y_test_pred)
print("MAPE:", mape)

In [ ]:
# Work Dataset
# RMSE: 1.05554 (FrequencyEncoder)
# RMSE: 1.05597 (OneHotEncoder)

In [ ]:
# Home Dataset
# RMSE: 1.04541 (FrequencyEncoder)
# RMSE: 1.04536 (OneHot Encode): Submit Result = 1.0567

In [ ]:
# Create final predictions
df_solve_cleaned = clean_dataframe(df_solve, impute_num_nulls = True, fill_cat_nulls=False)
df_solve_prepared = transform_dataframe(df_solve_cleaned)
X_solve_prepared, X_solve_prepared_df = pipeline_transform(df_solve_prepared, pipeline, cat_encoder = cat_encoder_type)

y_solve = model.predict(X_solve_prepared)

df_y_solve = pd.DataFrame(y_solve, columns=['y'])
df_final = pd.merge(df_solve_id, df_y_solve, left_index=True, right_index=True, how='inner')
df_final.to_csv('Predictions_XGBoost.csv', index=False)

df_final['y'].hist(bins=100)